# Array Copying and Views

Understand when NumPy returns **views** vs **copies**, how `view()` and `copy()` behave, and how to check **memory sharing** between arrays.


## Assignment vs View vs Copy

- **Assignment** (`b = a`) does **not** duplicate data. `b` points to the **same** array object as `a`.
- **View** (e.g., slicing, `reshape` when possible, `ravel` when possible, `.T`) creates a **new array object** that **shares** the same underlying data buffer.
- **Copy** (e.g., `a.copy()`, `flatten()`, most advanced indexing) creates a **new array object with its own data**.


In [ ]:
import numpy as np

a = np.arange(10)
b_assign = a            # assignment (same object)
b_slice = a[2:7]        # view when slicing
b_copy  = a.copy()      # deep copy

print('id(a)       :', id(a))
print('id(b_assign):', id(b_assign), '(same object?)', a is b_assign)
print('id(b_slice) :', id(b_slice),  '(different object?)', a is b_slice)
print('id(b_copy)  :', id(b_copy),   '(different object?)', a is b_copy)

# Mutate original and observe propagation
a[3] = 999
print('\nAfter a[3]=999')
print('a       :', a)
print('b_assign:', b_assign, '   # same object -> reflects change')
print('b_slice :', b_slice,  '       # view -> reflects change')
print('b_copy  :', b_copy,   '        # copy -> unchanged')

## `view()` vs `copy()`

- `a.view()` returns a **new view** of the same data buffer (no data copy).  
  Useful for changing metadata like dtype or creating a new view object.
- `a.copy()` returns a **deep copy** with independent memory.


In [ ]:
x = np.arange(6, dtype=np.int32).reshape(2,3)
v = x.view()         # view of x
c = x.copy()         # deep copy

print('x.base is None? ', x.base is None)
print('v.base is x?    ', v.base is x)
print('c.base is None? ', c.base is None)

# Mutate x and observe
x[0,0] = -1
print('\nAfter x[0,0] = -1')
print('x:\n', x)
print('view v shares data:\n', v)
print('copy c independent:\n', c)

### Views from common operations

- **Slicing** → view  
- **Transpose** (`.T`) → view  
- **`reshape()`** → view when possible (contiguity + compatibility)  
- **`ravel()`** → view when possible; **`flatten()`** → copy  
- **Advanced indexing** (integer/boolean/fancy) → copy


In [ ]:
y = np.arange(12).reshape(3,4)
s  = y[:, 1:3]           # slice -> view
t  = y.T                 # transpose -> view
rv = y.ravel()           # often a view
fl = y.flatten()         # always a copy

mask = y % 2 == 0
adv = y[mask]            # boolean indexing -> copy

print('s.base is y? ', s.base is y)
print('t.base is y? ', t.base is y)
print('rv owns data? ', rv.flags['OWNDATA'])   # False -> view
print('fl owns data? ', fl.flags['OWNDATA'])   # True  -> copy

print('adv owns data? ', adv.flags['OWNDATA']) # True  -> copy

## Memory Sharing Between Arrays

Two helpful utilities:
- `np.shares_memory(a, b)` → **definite** sharing (True/False).
- `np.may_share_memory(a, b)` → **conservative** check (may be True even if not).
Also useful flags/attributes:
- `arr.flags['OWNDATA']` — whether array **owns** its data
- `arr.base` — reference to the **owner** of the memory if it is a view


In [ ]:
p = np.arange(16)
q = p.reshape(4,4)          # likely a view
r = p.copy()

print('shares_memory(p, q):', np.shares_memory(p, q))
print('shares_memory(p, r):', np.shares_memory(p, r))

print('may_share_memory(p, q):', np.may_share_memory(p, q))
print('may_share_memory(p, r):', np.may_share_memory(p, r))

print('\nOwnership flags:')
print('p OWNDATA:', p.flags['OWNDATA'])
print('q OWNDATA:', q.flags['OWNDATA'], ' | q.base is p ?', q.base is p)
print('r OWNDATA:', r.flags['OWNDATA'], ' | r.base:', r.base)

## Subtle Cases and Pitfalls

- **Non-contiguous arrays** (e.g., slicing with a step, `T` on non-square) may force `reshape` to return a **copy**.  
- **Mixed operations**: chaining views then applying advanced indexing usually yields a **copy**.  
- **Changing dtype with `view(dtype=...)`**: creates a new view interpreting the **same bytes** with a different dtype (use carefully).


In [ ]:
# Non-contiguous view then reshape
z = np.arange(20)
step_view = z[::2]              # strided view, non-contiguous
try_view = step_view.reshape(2,5)  # may require a copy depending on layout
print('step_view OWNDATA:', step_view.flags['OWNDATA'])
print('try_view OWNDATA:', try_view.flags['OWNDATA'])

# dtype reinterpretation with view (advanced usage)
raw = np.arange(4, dtype=np.uint8)
as_uint32 = raw.view(np.uint32)  # reinterpret bytes as uint32
print('\nRaw bytes:', raw)
print('Reinterpreted as uint32 (platform-endian!):', as_uint32)